In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
dataset = pd.read_csv('train.csv')
# Drop irrelevant data labels into the training
dataset = dataset.drop(['PassengerId', 'Name'], axis=1)
# Convert number to float type
dataset['CryoSleep'] = dataset['CryoSleep'].astype(float)
dataset['VIP'] = dataset['VIP'].astype(float)
dataset['Transported'] = dataset['Transported'].astype(float)
label_encoders = {}
# Encoding categorical columns ('HomePlanet', 'Cabin', 'Destination') to numeric values
for column in ['HomePlanet', 'Cabin', 'Destination']:
    le = LabelEncoder()
    dataset[column] = le.fit_transform(dataset[column]).astype(float)
    label_encoders[column] = le
dataset.fillna(0, inplace=True)
dataset.info()
# Splitting the dataset into features and target variable
X = dataset.drop("Transported", axis=1).values
y = dataset["Transported"].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   float64
 1   CryoSleep     8693 non-null   float64
 2   Cabin         8693 non-null   float64
 3   Destination   8693 non-null   float64
 4   Age           8693 non-null   float64
 5   VIP           8693 non-null   float64
 6   RoomService   8693 non-null   float64
 7   FoodCourt     8693 non-null   float64
 8   ShoppingMall  8693 non-null   float64
 9   Spa           8693 non-null   float64
 10  VRDeck        8693 non-null   float64
 11  Transported   8693 non-null   float64
dtypes: float64(12)
memory usage: 815.1 KB


In [3]:
# Defining a custom neural network class that inherits from nn.Module
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 128)
        self.layer4 = nn.Linear(128, 64)
        self.layer5 = nn.Linear(64, 16)
        self.layer6 = nn.Linear(16, 4)
        self.layer7 = nn.Linear(4, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.layer1(x))
        out = self.relu(self.layer2(out))
        out = self.relu(self.layer3(out))
        out = self.relu(self.layer4(out))
        out = self.relu(self.layer5(out))
        out = self.relu(self.layer6(out))
        out = torch.sigmoid(self.layer7(out))
        return out

In [4]:
model = NeuralNet(X_train.shape[1]).to(device)
print(model)
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
# Training the model
num_epochs = 2000
for epoch in range(num_epochs):
    # Training phase
    model.train()
    outputs = model(X_train)
    print(outputs.squeeze().min(), outputs.squeeze().max())
    loss = criterion(outputs.squeeze(), y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_preds = outputs.squeeze().detach().cpu().numpy() > 0.5
    train_acc = accuracy_score(y_train.cpu().numpy(), train_preds)
    # Evaluation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs.squeeze(), y_val)
        val_preds = val_outputs.squeeze().cpu().numpy() > 0.5
        val_acc = accuracy_score(y_val.cpu().numpy(), val_preds)
    print(
        f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {loss.item()}, Training Accuracy: {train_acc}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_acc}")

NeuralNet(
  (layer1): Linear(in_features=11, out_features=1024, bias=True)
  (layer2): Linear(in_features=1024, out_features=512, bias=True)
  (layer3): Linear(in_features=512, out_features=128, bias=True)
  (layer4): Linear(in_features=128, out_features=64, bias=True)
  (layer5): Linear(in_features=64, out_features=16, bias=True)
  (layer6): Linear(in_features=16, out_features=4, bias=True)
  (layer7): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
)
tensor(0.0013, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.9820, device='cuda:0', grad_fn=<MaxBackward1>)
Epoch 1/2000, Training Loss: 0.6987248063087463, Training Accuracy: 0.545010066148979, Validation Loss: 0.6767534613609314, Validation Accuracy: 0.5842438182863715
tensor(0.0008, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.9637, device='cuda:0', grad_fn=<MaxBackward1>)
Epoch 2/2000, Training Loss: 0.6852712035179138, Training Accuracy: 0.5721886683922922, Validation Loss: 0.6638474464416504, Validation 

In [7]:
# Importing the test dataset from the CSV
print(f"Validation Accuracy: {val_acc}")
dataset = pd.read_csv('test.csv')
dataset = dataset.drop(['PassengerId', 'Name'], axis=1)
dataset['CryoSleep'] = dataset['CryoSleep'].astype(float)
dataset['VIP'] = dataset['VIP'].astype(float)
dataset['Transported'] = dataset['Transported'].astype(float)
label_encoders = {}
for column in ['HomePlanet', 'Cabin', 'Destination']:
    le = LabelEncoder()
    dataset[column] = le.fit_transform(dataset[column]).astype(float)
    label_encoders[column] = le
dataset.fillna(0, inplace=True)
dataset.info()
X = dataset.drop("Transported", axis=1).values
X_train = torch.tensor(X, dtype=torch.float32).to(device)
train_preds = model(X_train)
y_pred_binary = [True if prob > 0.5 else False for prob in train_preds]
sample_submission_df = pd.read_csv('sample_submission.csv')
sample_submission_df['Transported'] = y_pred_binary
sample_submission_df.to_csv('sample_submission.csv', index=False)
sample_submission_df.head()

Validation Accuracy: 0.7849338700402531
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4277 non-null   float64
 1   CryoSleep     4277 non-null   float64
 2   Cabin         4277 non-null   float64
 3   Destination   4277 non-null   float64
 4   Age           4277 non-null   float64
 5   VIP           4277 non-null   float64
 6   RoomService   4277 non-null   float64
 7   FoodCourt     4277 non-null   float64
 8   ShoppingMall  4277 non-null   float64
 9   Spa           4277 non-null   float64
 10  VRDeck        4277 non-null   float64
 11  Transported   4277 non-null   float64
dtypes: float64(12)
memory usage: 401.1 KB


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
